# Analisador Audio Texto

Caderno para conversão e análise de audio.

Pré requisitos:

Instalar o ffmpeg para conversão de formatos de áudio.

Pode-se instalar pelo Homebrew por meio do comando: brew install ffmpeg.

Instalar a biblioteca vosk: pip install vosk

Baixar o modelo do vosk em português no site:
https://alphacephei.com/vosk/models
Arquivo: vosk-model-small-pt-0.3


In [83]:
# Importação das bibliotecas
import os

import subprocess
import sys

from vosk import Model, KaldiRecognizer, SetLogLevel

import json

import pandas as pd

import IPython

import time

from tqdm.notebook import tqdm
import ipywidgets as widgets

In [94]:
# Ler vários formatos de arquivo de áudio de uma pasta.
#pasta = os.getcwd() + r'/Audio' + r'/'
pasta = '/Users/andreluiz/Documents/projetos/analisador-audio-texto/Audio/'

lista_arquivos = os.listdir(pasta)
print(f"Quantidade de aquivos na pasta {pasta}: ", len(lista_arquivos))
print(lista_arquivos)

Quantidade de aquivos na pasta /Users/andreluiz/Documents/projetos/analisador-audio-texto/Audio/:  2
['PTT-20200304-WA0005.opus', 'PTT-20200304-WA0006.opus']


## 1. Leitura e Transcrição dos áudios

In [95]:
# Inicializar variáveis e modelo

SAMPLE_RATE = 16000

SetLogLevel(0)

model = Model('models/modelos/vosk-model-small-pt-0.3')
rec = KaldiRecognizer(model, SAMPLE_RATE)
lista_transcricoes = []

# Ler cada arquivo e transcrever
inicio = time.time()

for arquivo in tqdm(lista_arquivos):    
    print(f"Transcrevendo arquivo: {arquivo}")
    with subprocess.Popen(["ffmpeg", "-loglevel", "quiet", "-i",
                            pasta + arquivo,
                            "-ar", str(SAMPLE_RATE) , "-ac", "1", "-f", "s16le", "-"],
                            stdout=subprocess.PIPE) as process:
        transcricao = ' '
        dic_transcricao = {}
  

        while True:
            data = process.stdout.read(4000)
            if len(data) == 0:
                break
            if rec.AcceptWaveform(data):
                dic_transcricao = json.loads(rec.Result())
                transcricao = transcricao + ' ' + dic_transcricao['text']

        dic_transcricao = json.loads(rec.FinalResult())
        transcricao = transcricao + ' ' + dic_transcricao['text']
        lista_transcricoes.append(transcricao)

fim = time.time()

print("Tempo de Execução: ",(fim - inicio), " segundos")
print("Tempo de Execução: ",(fim - inicio)/60, " minutos")
print("Total de Arquivos Transcritos: ",len(lista_transcricoes) )

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from models/modelos/vosk-model-small-pt-0.3/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from models/modelos/vosk-model-small-pt-0.3/HCLr.fst models/modelos/vosk-model-small-pt-0.3/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo models/modelos/vosk-model-small-pt-0.3/word_boundary.int


  0%|          | 0/2 [00:00<?, ?it/s]

Transcrevendo arquivo: PTT-20200304-WA0005.opus
Transcrevendo arquivo: PTT-20200304-WA0006.opus
Tempo de Execução:  17.619145154953003  segundos
Tempo de Execução:  0.2936524192492167  minutos
Total de Arquivos Transcritos:  2


In [96]:
# Cria um DataFrame com as transcrições
# O nome do arquivo é o índice do Dataframe, mas depois esse índice é convertido em coluna
# Acrescenta-se uma nova coluna com o caminho do arquivo
df_audio_transcricoes = []
df_audio_transcricoes =  pd.DataFrame(lista_transcricoes, index=lista_arquivos, columns=['transcricao'])
df_audio_transcricoes

,transcricao
PTT-20200304-WA0005.opus,pa já sabe estou saindo pra ir pra lá o lá p...
PTT-20200304-WA0006.opus,mensagem eu tenho longo de dois chega lá voc...


# 2. Filtros

## 2.1 Filtros pelo nome do arquivo

In [98]:
# Realizar filtros pelo índice com nome do arquivo
arquivo_selecionado = 'PTT-20200304-WA0006.opus'

print(df_audio_transcricoes['transcricao'].loc[arquivo_selecionado])


IPython.display.Audio(pasta + arquivo_selecionado)



  mensagem eu tenho longo de dois chega lá você vai começar com espremer te passo nome a passo agora o nome que eu tenho aqui não há como onde quer que querem quente aí eu faço chega lá você lá então após


## 2.2 Filtro por Palavras-Chave

Realizar filtro por palavras-chaves em todos as transcrições a partir de uma planilhas com as palavras-chave

In [104]:
# Realizar filtro por palavras-chaves em todos as transcrições
lista_palavras_chaves = []
palavra_chave = []

# Lista de palavras chaves
#lista_palavras_chaves = ['jesus abençoado', 'família']

# Lista de palavras-chave a partir de um arquivo excel
df_palavras_chaves = pd.read_excel('palavras_chave.xlsx')
lista_palavras_chaves = list(df_palavras_chaves['Termo'])

# Transforma a lista em uma string separada por | que representa o operador OR.
# Depois transforma a string para miníscula
palavra_chave = '|'.join(lista_palavras_chaves)
palavra_chave = palavra_chave.lower()

serie_resultado = df_audio_transcricoes['transcricao'].str.contains(palavra_chave)
serie_resultado

PTT-20200304-WA0005.opus    False
PTT-20200304-WA0006.opus     True
Name: transcricao, dtype: bool

In [105]:
# Mostra as palavras chave utilizadas
palavra_chave

'gerir|transferência|depósito|contrato|dossie|caixinha|acordo|família|jesus|jesus abençoado|mensagem'

In [106]:
# Filtros da palavra chave
# Podemos mostrar tanto os audios em que APARECEM (True) as palavras chaves como NAO APARECEM (False)
# Filtra as colunas que APARECEM as palavras chave (True). Para False colocar ~ antes
df_filtrado = []
df_filtrado = df_audio_transcricoes[serie_resultado]
df_filtrado

,transcricao
PTT-20200304-WA0006.opus,mensagem eu tenho longo de dois chega lá voc...


# 3. Exportação para Excel

In [107]:
# Prepara os Dataframes para exportação, transformando os índices em colunas e acrescentando o caminho do arquivo

df_saida_audio_transcricoes = []
df_saida_filtrado = []

df_saida_audio_transcricoes = df_audio_transcricoes.rename_axis('arquivo').reset_index()
df_saida_audio_transcricoes['caminho'] = pasta + df_saida_audio_transcricoes['arquivo']

if len(df_filtrado) == 0:
    df_saida_filtrado = pd.DataFrame({'mensagem' : ['Não econtrou ocorrências nas palavras chave'] })
else:
    df_saida_filtrado = df_filtrado.rename_axis('arquivo').reset_index()
    df_saida_filtrado['caminho'] = pasta + df_saida_filtrado['arquivo']

# Exporta os Dataframes para excel

saida = 'Saida/audios_transcritos.xlsx'

with pd.ExcelWriter(saida) as writer:
    df_saida_audio_transcricoes.to_excel(writer, index=False, sheet_name='audios_transcritos')
    df_saida_filtrado.to_excel(writer, index=False, sheet_name='audios_filtrados')
    df_palavras_chaves.to_excel(writer, index=False, sheet_name='palavras_chave_utilizadas')

print(f"Arquivo gerado: {saida}")

Arquivo gerado: Saida/audios_transcritos.xlsx
